In [1]:
from ase.build import mx2
from gpaw import GPAW, PW, FermiDirac
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import progressbar
from gpaw.utilities.ps2ae import PS2AE
from ase.units import Bohr,Hartree
%matplotlib inline

structure = mx2(formula='MoS2', kind='2H', a=3.184, thickness=3.127,size=(1, 1, 1), vacuum=3.5)
structure.pbc = (1, 1, 0)
calc = GPAW(mode=PW(400),xc='LDA',setups={'Mo': '6'}, parallel={'kpt': 1},kpts={'size': (8,8,1), 'gamma': True},occupations=FermiDirac(0.01))

structure.set_calculator(calc)
structure.get_potential_energy()

calc.write('MoS2_gs.gpw', 'all')


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  1.3.0
 |___|_|             

User:   titan@Titan
Date:   Wed Apr 11 17:07:54 2018
Arch:   x86_64
Pid:    3533
Python: 2.7.14
gpaw:   /home/titan/anaconda2/lib/python2.7/site-packages/gpaw
_gpaw:  /home/titan/anaconda2/lib/python2.7/site-packages/_gpaw.so
ase:    /home/titan/.local/lib/python2.7/site-packages/ase (version 3.15.0)
numpy:  /home/titan/anaconda2/lib/python2.7/site-packages/numpy (version 1.14.0)
scipy:  /home/titan/anaconda2/lib/python2.7/site-packages/scipy (version 1.0.0)
units:  Angstrom and eV
cores:  1

Input parameters:
  kpts: {gamma: True,
         size: (8, 8, 1)}
  mode: {ecut: 400.0,
         name: pw}
  occupations: {name: fermi-dirac,
                width: 0.01}
  setups: {Mo: 6}

System changes: positions, numbers, cell, pbc, initial_charges, initial_magmoms 

Initialize ...

Mo-setup:
  name: Molybdenum
  id: 4748e35d57ecde6b74fb9d143c53017c
  Z: 42
  valence: 6
  core:

iter:  37  17:08:06  -5.54  -3.69   -24.097760    0             
iter:  38  17:08:06  -5.57  -3.87   -24.097763    0             
iter:  39  17:08:07  -5.46  -3.87   -24.097763    0             
iter:  40  17:08:07  -5.78  -3.97   -24.097764    0             
iter:  41  17:08:07  -5.54  -3.99   -24.097764    0             
iter:  42  17:08:08  -5.79  -4.11   -24.097765    0             
iter:  43  17:08:08  -5.97  -4.23   -24.097765    0             
iter:  44  17:08:08  -6.16  -4.24   -24.097766    0             
iter:  45  17:08:08  -6.05  -4.13   -24.097766    0             
iter:  46  17:08:09  -6.14  -4.39   -24.097767    0             
iter:  47  17:08:09  -6.23  -4.47   -24.097767    0             
iter:  48  17:08:09  -6.29  -4.43   -24.097767    0             
iter:  49  17:08:10  -6.43  -4.65   -24.097767    0             
iter:  50  17:08:10  -6.66  -4.65   -24.097767    0             
iter:  51  17:08:10  -6.54  -4.70   -24.097767    0             
iter:  52  17:08:10  -6.6

In [2]:
calc.diagonalize_full_hamiltonian(nbands=20)

Diagonalizing full Hamiltonian (20 lowest bands)
Matrix size (min, max): 1605, 1629
Approximate memory used per core to store H_GG, S_GG: 121.474 MB
Notice: Up to twice the amount of memory might be allocated
during diagonalization algorithm.
The least memory is required when the parallelization is purely
over states (bands) and not k-points, set GPAW(..., parallel={'kpt': 1}, ...).
17.2887420654s |----------------------------------------| Time: 13.803s


In [3]:
AE = PS2AE(structure.calc, h=0.1)

In [4]:
bands=range(calc.get_number_of_bands());nbands=len(bands)
ikpoints=calc.get_ibz_k_points();ink=ikpoints.shape[0]
kpoints=np.round(calc.get_bz_k_points(),8);nk=kpoints.shape[0]
kp_to_ikp=calc.get_bz_to_ibz_map()
ikp_to_kp={}
for i in range(ink): ikp_to_kp[i]=[]
for i in range(nk):ikp_to_kp[kp_to_ikp[i]].append(i)
for i in range(ink): ikp_to_kp[i]=np.array(ikp_to_kp[i])
Ef=calc.get_fermi_level()
volume=structure.get_volume()
e_kn = np.array([calc.get_eigenvalues(kpt=k)for k in range(len(calc.get_ibz_k_points()))])
e_kn=e_kn[:,bands]
e_kn=e_kn-Ef
f_kn=np.ones_like(e_kn)
f_kn[e_kn>0]=0
kn=[(k,n) for k,n in product(range(nk),range(nbands))]
ikn=[(k,n) for k,n in product(range(ink),range(nbands))]
kn_to_ink=np.zeros((nk,nbands),dtype=np.int)
for i in range(len(ikn)):
    ik,n=ikn[i]
    kn_to_ink[ikp_to_kp[ik],n]=i
q0=np.arange(len(kpoints))[np.sum(kpoints**2,axis=1)==0][0]

In [5]:
def find_kpoint(kpt):
    kpt[kpt<=-0.5]+=1;kpt[kpt>0.5]-=1
    return np.where(np.array([(kpoints[k]==kpt).all() for k in range(nk)]))[0][0]

In [ ]:
wf=[AE.get_wave_function(n,k) for k,n in ikn]

In [ ]:
rho=np.zeros((nk,nbands,nk,nbands,nk),dtype=np.complex)
bar=progressbar.ProgressBar()
for q in bar(range(nk)):
    exp=AE.gd.plane_wave(kpoints[q])
    for i1 in range(len(ikn)):
        for i2 in range(i1,len(ikn)):
            ik1,n1=ikn[i1];ik2,n2=ikn[i2];wf1=wf[i1];wf2=wf[i2]
            k1,k2=np.meshgrid(ikp_to_kp[ik1],ikp_to_kp[ik2])
            result=AE.gd.integrate(wf1,exp*wf2,hermitian=True)*Bohr**3
            rho[q,n1,k1,n2,k2]=result
            rho[q,n2,k2,n1,k1]=np.conj(result)
    

In [ ]:
def chi0_GGq(arg):
    G1,G2,q=arg
    result=np.zeros((omega.size),dtype=np.complex)
    rho_kpt1=find_kpoint(kpoints[q]+kpoints[G1])
    rho_kpt2=find_kpoint(-kpoints[q]-kpoints[G2])
    for k1 in range(nk):
        k2=find_kpoint(kpoints[k1]+kpoints[q])
        for n1,n2 in product(range(nbands),range(nbands)):
            k1=kp_to_ikp[k1];k2=kp_to_ikp[k2]
            term1=f_kn[k1,n1]-f_kn[k2,n2]
            term2=rho[rho_kpt1,n1,k1,n2,k2]*rho[rho_kpt2,n1,k1,n2,k2]
            term3=omega+1j*eta+e_kn[k1,n1]-e_kn[k2,n2]
            result+=term1*term2/term3
    return result

In [ ]:
import multiprocessing 
omega=np.linspace(0,5,1000);eta=0.01
bar=progressbar.ProgressBar()
chi0=np.zeros((nk,nk,nk,omega.size),dtype=np.complex)
for q in bar(range(nk)):
    pool = multiprocessing.Pool(processes=12)
    result=pool.map(chi0_GGq, [(G1,G2,q) for G1,G2 in product(range(nk),range(nk))])
    chi0[:,:,q]=np.array(result).reshape((nk,nk,omega.size))
    pool.close()
    pool.join()
chi0/=volume

In [ ]:
V=np.zeros((nk,nk,nk),dtype=np.complex)
for q in range(nk):
    for G in range(nk):
        if np.sum(np.abs(kpoints[G]+kpoints[q]))==0: V[q,G,G]=0
        else: V[q,G,G]=4*np.pi/(np.sum(np.abs(kpoints[G]+kpoints[q])**2))

In [ ]:
e_RPA=np.zeros((nk,nk,nk,omega.size),dtype=np.complex)
for q in range(nk):
    for w in range(omega.size):
        e_RPA[q,:,:,w]=np.eye(nk)-np.dot(V[q],chi0[q,:,:,w])

In [ ]:
W=np.zeros_like(e_RPA)
for q in range(nk):
    for w in range(omega.size):
        W[q,:,:,w]=np.dot(np.linalg.inv(e_RPA[q,:,:,w]),V[q])
W0=W[:,:,:,0]

In [ ]:
val_bands=[7,8];con_bands=[9,10]
nv=len(val_bands);nc=len(con_bands);nS=nk*nc*nv
H_exciton=np.zeros((nk,nc,nv,nk,nc,nv),dtype=np.complex)
for k1,k2 in product(range(nk),range(nk)):
    ik1=kp_to_ikp[k1];ik2=kp_to_ikp[k2]
    for c1,c2 in product(con_bands,con_bands):
        for v1,v2 in product(val_bands,val_bands):
            iv1=v1-val_bands[0];iv2=v2-val_bands[0];
            ic1=c1-con_bands[0];ic2=c2-con_bands[0];
            if k1==k2 and v1==v2 and c1==c2:
                H_exciton[k1,ic1,iv1,k2,ic2,iv2]+=e_kn[ik1,c1]-e_kn[ik1,v1]
            V_=np.sum(rho[:,v1,k1,c1,k1].conj()*V[q].diagonal()*rho[:,v2,k2,c2,k2])
            W_=np.sum(np.dot(rho[:,v1,k1,v2,k2][None,:],np.dot(W0[q],rho[:,c1,k1,c2,k2][:,None])))
            H_exciton[k1,ic1,iv1,k2,ic2,iv2]+=V_-W_

In [ ]:
E,D=np.linalg.eig(H_exciton.reshape(nS,nS))

In [ ]:
E[E.real>0]

In [ ]:
plt.hist(E[E.real>0].real,200)
plt.grid()

In [ ]:
H_trion=np.zeros((nk,nk,nc,nc,nv,nk,nk,nc,nc,nv),dtype=np.complex)
for k1,k2 in product(range(nk),range(nk)):
    kv=find_kpoint(kpoints[k1]+kpoints[k2])
    ik1=kp_to_ikp[k1];ik2=kp_to_ikp[k2];ikv=kp_to_ikp[kv]
    for c1,c2,v in product(con_bands,con_bands,val_bands):
        ic1=c1-con_bands[0];ic2=c2-con_bands[0];iv=v-val_bands[0]
        H_trion[k1,k2,ic1,ic2,iv,k1,k2,ic1,ic2,iv]=e_kn[ik1,c1]+e_kn[ik2,c2]-e_kn[ikv,v]

for k1,k2 in product(range(nk),range(nk)):
    for k1_,k2_ in product(range(nk),range(nk)):
        for c1,c2 in product(con_bands,con_bands):
            ic1=c1-con_bands[0];ic2=c2-con_bands[0];
            for c1_,c2_ in product(con_bands,con_bands):
                ic1_=c1_-con_bands[0];ic2_=c2_-con_bands[0];
                W1=np.sum(np.dot(rho[:,c1,k1,c1_,k1_][None,:].conj(),np.dot(W0[q],rho[:,c2,k2,c2_,k2_][:,None])))
                W2=np.sum(np.dot(rho[:,c2,k2,c2_,k2_][None,:].conj(),np.dot(W0[q],rho[:,c1,k1,c1_,k1_][:,None])))
                H_trion[k1,k2,ic1,ic2,range(nv),k1_,k2_,ic1_,ic2_,range(nv)]+=(W1-W2)
        kv=find_kpoint(kpoints[k1]+kpoints[k2])
        kv_=find_kpoint(kpoints[k1_]+kpoints[k2_])
        for c,c_ in product(con_bands,con_bands):
            for v,v_ in product(val_bands,val_bands):
                iv=v-val_bands[0];iv_=v_-val_bands[0];
                ic=c-con_bands[0];ic_=c_-con_bands[0];
                
                V_=np.sum(rho[:,v,kv,c,k1].conj()*V[q].diagonal()*rho[:,v_,kv_,c_,k1_])
                W_=np.sum(np.dot(rho[:,v,kv,v_,kv_][None,:],np.dot(W0[q],rho[:,c,k1,c_,k1_][:,None])))
                H_trion[k1,k2,ic,range(nc),iv,k1_,k2_,ic_,range(nc),iv_]=(V_-W_)
                
                V_=np.sum(rho[:,v,kv,c,k2].conj()*V[q].diagonal()*rho[:,v_,kv_,c_,k2_])
                W_=np.sum(np.dot(rho[:,v,kv,v_,kv_][None,:],np.dot(W0[q],rho[:,c,k2,c_,k2_][:,None])))
                H_trion[k1,k2,ic,range(nc),iv,k1_,k2_,ic_,range(nc),iv_]=(V_-W_)

In [ ]:
nSS=nk*nk*nc*nc*nv

In [ ]:
E=np.linalg.eigvals(H_trion.reshape(nSS,nSS))

In [ ]:
E=E[E.real>0.5];E=E[E.real<3]

In [ ]:
plt.hist(E.real,100)
plt.grid()